In [1]:

import sympy as sp
from sympy import symbols, pprint
from sympy import sin, cos, asin, acos, pi
from sympy import Matrix, simplify, Function, diff, Derivative, nsimplify
from sympy import lambdify, diag

import numpy as np

import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()


mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16

from IPython.display import display
sp.init_printing(use_latex=True);

In [2]:
%qtconsole

## 1 Viscous forces on a flexible arm

## 1.

In [3]:
(t, 
 theta_1, 
 theta_2, 
 theta_3, 
 l_1, 
 l_2, 
 l_3, 
 s) = symbols("""t, 
                 theta_1 
                 theta_2 
                 theta_3 
                 l_1 
                 l_2 
                 l_3
                 s""" , real = True)


theta_1 = Function('theta_1', real=True)(t)
theta_2 = Function('theta_2', real=True)(t)
theta_3 = Function('theta_3', real=True)(t)

In [4]:
def T(theta, x, y):
    """
    Function to return an arbitrary transformation matrix 
    This is for sympy symbolic calculation
    """
    return Matrix([[cos(theta), -sin(theta), x], 
                   [sin(theta), cos(theta), y],
                   [0, 0, 1]])
def sym_to_np(T):
    return np.array(T).astype(np.float64)

In [5]:
T_01 = T(theta_1, 0, 0)
T_12 = T(theta_2, l_1, 0)
T_23 = T(theta_3, l_2, 0)

In [6]:
FK1 = (T_01 * Matrix([[s], [0], [1]]))[:-1,0]
FK2 = simplify((T_01 * T_12 * Matrix([[s], [0], [1]]))[:-1,0])
FK3 = simplify((T_01 * T_12 * T_23 * Matrix([[s], [0], [1]]))[:-1,0])
J1 = FK1.jacobian([theta_1, theta_2, theta_3])
J2 = simplify(FK2.jacobian([theta_1, theta_2, theta_3]))
J3 = simplify(FK3.jacobian([theta_1, theta_2, theta_3]))
display(J1)
display(J2)
display(J3)


⎡-s⋅sin(θ₁(t))  0  0⎤
⎢                   ⎥
⎣s⋅cos(θ₁(t))   0  0⎦

⎡-l₁⋅sin(θ₁(t)) - s⋅sin(θ₁(t) + θ₂(t))  -s⋅sin(θ₁(t) + θ₂(t))  0⎤
⎢                                                               ⎥
⎣l₁⋅cos(θ₁(t)) + s⋅cos(θ₁(t) + θ₂(t))   s⋅cos(θ₁(t) + θ₂(t))   0⎦

⎡-l₁⋅sin(θ₁(t)) - l₂⋅sin(θ₁(t) + θ₂(t)) - s⋅sin(θ₁(t) + θ₂(t) + θ₃(t))  -l₂⋅si
⎢                                                                             
⎣l₁⋅cos(θ₁(t)) + l₂⋅cos(θ₁(t) + θ₂(t)) + s⋅cos(θ₁(t) + θ₂(t) + θ₃(t))   l₂⋅cos

n(θ₁(t) + θ₂(t)) - s⋅sin(θ₁(t) + θ₂(t) + θ₃(t))  -s⋅sin(θ₁(t) + θ₂(t) + θ₃(t))
                                                                              
(θ₁(t) + θ₂(t)) + s⋅cos(θ₁(t) + θ₂(t) + θ₃(t))   s⋅cos(θ₁(t) + θ₂(t) + θ₃(t)) 

⎤
⎥
⎦

## 2.

In [7]:
def draw_robot(ordered_list_of_transformations):
    """
    Draw the configuration of the three-link jumper.
    """
    ax_lw = 3
    link_lw = 1

    # homogeneous unit vectors 
    x = np.array([[1, 0, 1]]).T 
    y = np.array([[0, 1, 1]]).T
    origin = np.array([[0, 0, 1]]).T

    prev_origin = np.array([[0, 0, 1]]).T    
    current_transformation = np.eye(3)

    plt.clf()
    ax = plt.gca()  # get current axis

    # now we plot
    plt.plot([prev_origin[0][0]], [prev_origin[1][0]], '-ko',linewidth=link_lw)
    plt.plot([prev_origin[0][0], x[0][0]], [prev_origin[1][0], x[1][0]], '-ro', linewidth=ax_lw)
    plt.plot([prev_origin[0][0], y[0][0]], [prev_origin[1][0], y[1][0]], '-go', linewidth=ax_lw)


    # loop the transforms in order of T_01, T_12, ... , T_N-1N
    for k, transform in enumerate(ordered_list_of_transformations):

        # update the transformation
        current_transformation = current_transformation @ transform
        new_origin = current_transformation @ origin
        new_x = current_transformation @ x
        new_y = current_transformation @ y

        # now we plot
        plt.plot([prev_origin[0][0], new_origin[0][0]], 
                 [prev_origin[1][0], new_origin[1][0]], 
                 '-ko',linewidth=link_lw)
        
        plt.plot([new_origin[0][0], new_x[0][0]], [new_origin[1][0], new_x[1][0]], '-r', linewidth=ax_lw)
        plt.plot([new_origin[0][0], new_y[0][0]], [new_origin[1][0], new_y[1][0]], '-g', linewidth=ax_lw)


#         ax.annotate('{'+str(k+1)+'}', xy=new_origin[0:2][0], xytext=new_origin[0:2][0])

        # lastly update the current_origin to the new_origin
        prev_origin = new_origin

    # lastly set the axes to be equal and appropriate
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')    

In [8]:
linklength = 1

time = np.linspace(0,15,100)
theta1 = list(np.pi/2*np.sin(time))
A = -np.pi/6
phi = np.pi/4
b = 3
psy = -np.pi/8
theta2 = list(A*np.sin(time+phi)**b+psy)
A3 = -3*np.pi/8
phi3 = np.pi/4
b3 = 1
psy3 = -np.pi/6
theta3 = list(A3*np.sin(time+phi3)**b3+psy3)

T_01_real = lambdify(theta_1, T_01)
T_12_real = lambdify([theta_2, l_1], T_12)
T_23_real = lambdify([theta_3, l_2], T_23)
T_34_real = sym_to_np(T(0, linklength, 0))

for theta1_current, theta2_current, theta3_current in zip(theta1, theta2, theta3):
    T01 = T_01_real(theta1_current)
    T12 = T_12_real(theta2_current,linklength)
    T23 = T_23_real(theta3_current,linklength)
    
    draw_robot([T01, T12, T23, T_34_real])
    
    sc = 25
    plt.axis([-5,5, -5, 5])
    plt.draw()
    plt.pause(0.01)

## 3.

In [9]:
dtime = np.gradient(time)
dtheta1_time = np.gradient(theta1)/dtime
dtheta2_time = np.gradient(theta2)/dtime
dtheta3_time = np.gradient(theta3)/dtime

vel1_lamb = lambdify([s, l_1, l_2, l_3, theta_1, theta_2, theta_3], J1)
vel2_lamb = lambdify([s, l_1, l_2, l_3, theta_1, theta_2, theta_3], J2)
vel3_lamb = lambdify([s, l_1, l_2, l_3, theta_1, theta_2, theta_3], J3)
pos1_lamb = lambdify([s, l_1, l_2, l_3, theta_1, theta_2, theta_3], FK1)
pos2_lamb = lambdify([s, l_1, l_2, l_3, theta_1, theta_2, theta_3], FK2)
pos3_lamb = lambdify([s, l_1, l_2, l_3, theta_1, theta_2, theta_3], FK3)

fig = plt.figure(2,figsize = (8,8))
for theta1_current, theta2_current, theta3_current, v1, v2, v3 in zip(theta1, theta2, theta3, dtheta1_time, dtheta2_time, dtheta3_time):
    T01 = T_01_real(theta1_current)
    T12 = T_12_real(theta2_current,linklength)
    T23 = T_23_real(theta3_current,linklength)
    
    draw_robot([T01, T12, T23, T_34_real])
    
    for ss in np.linspace(0.1, 1, 10):
        
        vel1 = -vel1_lamb(ss*linklength, 
                         linklength,
                         linklength,
                         linklength,
                         theta1_current,
                         theta2_current,
                         theta3_current)*Matrix([[v1], [v2], [v3]])
        pos1 = pos1_lamb(ss*linklength, 
                         linklength,
                         linklength,
                         linklength,
                         theta1_current,
                         theta2_current,
                         theta3_current)
        
        # print(pos1)
        plt.quiver(float(pos1[0]), float(pos1[1]), float(vel1[0]), float(vel1[1]))
        
        vel2 = -vel2_lamb(ss*linklength, 
                         linklength,
                         linklength,
                         linklength,
                         theta1_current,
                         theta2_current,
                         theta3_current)*Matrix([[v1], [v2], [v3]])
        pos2 = pos2_lamb(ss*linklength, 
                         linklength,
                         linklength,
                         linklength,
                         theta1_current,
                         theta2_current,
                         theta3_current)
        
        # print(pos1)
        plt.quiver(float(pos2[0]), float(pos2[1]), float(vel2[0]), float(vel2[1]))
        
        vel3 = -vel3_lamb(ss*linklength, 
                         linklength,
                         linklength,
                         linklength,
                         theta1_current,
                         theta2_current,
                         theta3_current)*Matrix([[v1], [v2], [v3]])
        pos3 = pos3_lamb(ss*linklength, 
                         linklength,
                         linklength,
                         linklength,
                         theta1_current,
                         theta2_current,
                         theta3_current)
        
        # print(pos1)
        plt.quiver(float(pos3[0]), float(pos3[1]), float(vel3[0]), float(vel3[1]))
    
    sc = 25
    plt.axis([-5,5, -5, 5])
    plt.draw()
    plt.pause(0.01)

## 4.

In [10]:
JT1_lamb = lambdify([s, l_1, l_2, l_3, theta_1, theta_2, theta_3], J1.T)
JT2_lamb = lambdify([s, l_1, l_2, l_3, theta_1, theta_2, theta_3], J2.T)
JT3_lamb = lambdify([s, l_1, l_2, l_3, theta_1, theta_2, theta_3], J3.T)

torque1 = [0]*len(time)
torque2 = [0]*len(time)
torque3 = [0]*len(time)
i = 0

fig = plt.figure(3, figsize = (15,15))
for theta1_current, theta2_current, theta3_current, v1, v2, v3 in zip(theta1, theta2, theta3, dtheta1_time, dtheta2_time, dtheta3_time):
    for ss in np.linspace(0.1, 1, 10):  
        vel1 = -vel1_lamb(ss*linklength, 
                         linklength,
                         linklength,
                         linklength,
                         theta1_current,
                         theta2_current,
                         theta3_current)*Matrix([[v1], [v2], [v3]])
        JT1 = JT1_lamb(ss*linklength, 
                         linklength,
                         linklength,
                         linklength,
                         theta1_current,
                         theta2_current,
                         theta3_current)
        Torq1 = JT1@vel1
        torque1[i] = Torq1[0]
        
        vel2 = -vel2_lamb(ss*linklength, 
                         linklength,
                         linklength,
                         linklength,
                         theta1_current,
                         theta2_current,
                         theta3_current)*Matrix([[v1], [v2], [v3]])
        JT2 = JT2_lamb(ss*linklength, 
                         linklength,
                         linklength,
                         linklength,
                         theta1_current,
                         theta2_current,
                         theta3_current)
        Torq2 = JT2@vel2
        torque2[i] = Torq2[1]
        
        vel3 = -vel3_lamb(ss*linklength, 
                         linklength,
                         linklength,
                         linklength,
                         theta1_current,
                         theta2_current,
                         theta3_current)*Matrix([[v1], [v2], [v3]])
        JT3 = JT3_lamb(ss*linklength, 
                         linklength,
                         linklength,
                         linklength,
                         theta1_current,
                         theta2_current,
                         theta3_current)
        Torq3 = JT3@vel3
        torque3[i] = Torq3[1]
    i+=1
        
plt.plot(time,torque1, 'r')
plt.plot(time,torque2, 'b')
plt.plot(time,torque3, 'k')
plt.xlabel("time")
plt.ylabel('torques')
plt.legend(['torque at joint 1','torque at joint 2','torque at joint 3'])
                                         

## 5.

In [11]:
linklength = 1

time = np.linspace(0,2*np.pi,100)
theta1 = list(np.pi/2*np.sin(time))

PreviousForce = 0
NetForce = [0]*len(time)
i = 0
fig = plt.figure(4, figsize = (15,15))
for theta1_current in theta1:
    for ss in np.linspace(0.1, 1, 10):  
         vel1 = -vel1_lamb(ss*linklength, 
                         linklength,
                         0,
                         0,
                         theta1_current,
                         0,
                         0)*Matrix([[v1], [0], [0]])
         Force1 = vel1[0]
         NetForce[i] = Force1+PreviousForce
         PreviousForce = vel1[0]
    i += 1
plt.plot(time,NetForce, 'r')
plt.xlabel('time')
plt.ylabel('NetForce')
display(NetForce[-1])

Very small, very close to 0

## 2 A model of standing

In [12]:
(t, 
 theta_1, 
 theta_2, 
 theta_3, 
 m_1, 
 m_2, 
 m_3, 
 L,
 g) = symbols("""t, 
                 theta_1 
                 theta_2 
                 theta_3 
                 m_1 
                 m_2 
                 m_3
                 L
                 g""" , real = True)


theta_1 = Function('theta_1', real=True)(t)
theta_2 = Function('theta_2', real=True)(t)
theta_3 = Function('theta_3', real=True)(t)

In [13]:
def T(theta, x, y):
    """
    Function to return an arbitrary transformation matrix 
    This is for sympy symbolic calculation
    """
    return Matrix([[cos(theta), -sin(theta), x], 
                   [sin(theta), cos(theta), y],
                   [0, 0, 1]])
def sym_to_np(T):
    return np.array(T).astype(np.float64)

In [14]:
T_01 = T(theta_1, 0, 0)
T_12 = T(theta_2, 0, L)
T_23 = T(theta_3, 0, L)

## 1.

In [15]:
FK1 = (T_01 * Matrix([[L], [0], [1]]))[:-1,0]
FK2 = simplify((T_01 * T_12 * Matrix([[L], [0], [1]]))[:-1,0])
FK3 = simplify((T_01 * T_12 * T_23 * Matrix([[L], [0], [1]]))[:-1,0])
display(FK1)
display(FK2)
display(FK3)

⎡L⋅cos(θ₁(t))⎤
⎢            ⎥
⎣L⋅sin(θ₁(t))⎦

⎡L⋅(-sin(θ₁(t)) + cos(θ₁(t) + θ₂(t)))⎤
⎢                                    ⎥
⎣L⋅(sin(θ₁(t) + θ₂(t)) + cos(θ₁(t))) ⎦

⎡L⋅(-sin(θ₁(t) + θ₂(t)) - sin(θ₁(t)) + cos(θ₁(t) + θ₂(t) + θ₃(t)))⎤
⎢                                                                 ⎥
⎣L⋅(sin(θ₁(t) + θ₂(t) + θ₃(t)) + cos(θ₁(t) + θ₂(t)) + cos(θ₁(t))) ⎦

## 2.

In [16]:
J1 = FK1.jacobian([theta_1, theta_2, theta_3])
J2 = simplify(FK2.jacobian([theta_1, theta_2, theta_3]))
J3 = simplify(FK3.jacobian([theta_1, theta_2, theta_3]))
display(J1)
display(J2)
display(J3)

⎡-L⋅sin(θ₁(t))  0  0⎤
⎢                   ⎥
⎣L⋅cos(θ₁(t))   0  0⎦

⎡-L⋅(sin(θ₁(t) + θ₂(t)) + cos(θ₁(t)))  -L⋅sin(θ₁(t) + θ₂(t))  0⎤
⎢                                                              ⎥
⎣L⋅(-sin(θ₁(t)) + cos(θ₁(t) + θ₂(t)))  L⋅cos(θ₁(t) + θ₂(t))   0⎦

⎡-L⋅(sin(θ₁(t) + θ₂(t) + θ₃(t)) + cos(θ₁(t) + θ₂(t)) + cos(θ₁(t)))  -L⋅(sin(θ₁
⎢                                                                             
⎣L⋅(-sin(θ₁(t) + θ₂(t)) - sin(θ₁(t)) + cos(θ₁(t) + θ₂(t) + θ₃(t)))  L⋅(-sin(θ₁

(t) + θ₂(t) + θ₃(t)) + cos(θ₁(t) + θ₂(t)))  -L⋅sin(θ₁(t) + θ₂(t) + θ₃(t))⎤
                                                                         ⎥
(t) + θ₂(t)) + cos(θ₁(t) + θ₂(t) + θ₃(t)))  L⋅cos(θ₁(t) + θ₂(t) + θ₃(t)) ⎦

## 3.

In [17]:
J = Matrix.vstack(J1,J2,J3)
J

⎡                          -L⋅sin(θ₁(t))                                      
⎢                                                                             
⎢                          L⋅cos(θ₁(t))                                       
⎢                                                                             
⎢              -L⋅(sin(θ₁(t) + θ₂(t)) + cos(θ₁(t)))                           
⎢                                                                             
⎢              L⋅(-sin(θ₁(t)) + cos(θ₁(t) + θ₂(t)))                           
⎢                                                                             
⎢-L⋅(sin(θ₁(t) + θ₂(t) + θ₃(t)) + cos(θ₁(t) + θ₂(t)) + cos(θ₁(t)))  -L⋅(sin(θ₁
⎢                                                                             
⎣L⋅(-sin(θ₁(t) + θ₂(t)) - sin(θ₁(t)) + cos(θ₁(t) + θ₂(t) + θ₃(t)))  L⋅(-sin(θ₁

               0                                          0              ⎤
                                                       

## 4.

In [18]:
m = diag(m_1,m_1,m_2,m_2,m_3,m_3)
M = J.T@m@J
M

⎡                                                2       2           2       2
⎢                                               L ⋅m₁⋅sin (θ₁(t)) + L ⋅m₁⋅cos 
⎢                                                                             
⎢ 2                                                            2              
⎢L ⋅m₂⋅(sin(θ₁(t) + θ₂(t)) + cos(θ₁(t)))⋅sin(θ₁(t) + θ₂(t)) + L ⋅m₂⋅(-sin(θ₁(t
⎢                                                                             
⎢                                                                             
⎣                                                                             

           2                                     2    2                       
(θ₁(t)) + L ⋅m₂⋅(sin(θ₁(t) + θ₂(t)) + cos(θ₁(t)))  + L ⋅m₂⋅(-sin(θ₁(t)) + cos(
                                                                              
                                               2                              
)) + cos(θ₁(t) + θ₂(t)))⋅cos(θ₁(t) + θ₂(t)) + L ⋅m₃

## 5.

In [19]:
M_real = lambdify([theta_1,theta_2,theta_3], M)
M_real(np.pi/2,0,0)

array([[1.0*L**2*m_1 + 2.0*L**2*m_2 + 5.0*L**2*m_3,
        1.0*L**2*m_2 + 3.0*L**2*m_3, 1.0*L**2*m_3],
       [1.0*L**2*m_2 + 3.0*L**2*m_3, 1.0*L**2*m_2 + 2.0*L**2*m_3,
        1.0*L**2*m_3],
       [1.0*L**2*m_3, 1.0*L**2*m_3, 1.0*L**2*m_3]], dtype=object)

## 6.

In [20]:
G1 = J1.T*Matrix([[0],[-m_1*g]])
G2 = J2.T*Matrix([[0],[-m_2*g]])
G3 = J3.T*Matrix([[0],[-m_3*g]])
display(G1)
display(G2)
display(G3)

⎡-L⋅g⋅m₁⋅cos(θ₁(t))⎤
⎢                  ⎥
⎢        0         ⎥
⎢                  ⎥
⎣        0         ⎦

⎡-L⋅g⋅m₂⋅(-sin(θ₁(t)) + cos(θ₁(t) + θ₂(t)))⎤
⎢                                          ⎥
⎢        -L⋅g⋅m₂⋅cos(θ₁(t) + θ₂(t))        ⎥
⎢                                          ⎥
⎣                    0                     ⎦

⎡-L⋅g⋅m₃⋅(-sin(θ₁(t) + θ₂(t)) - sin(θ₁(t)) + cos(θ₁(t) + θ₂(t) + θ₃(t)))⎤
⎢                                                                       ⎥
⎢      -L⋅g⋅m₃⋅(-sin(θ₁(t) + θ₂(t)) + cos(θ₁(t) + θ₂(t) + θ₃(t)))       ⎥
⎢                                                                       ⎥
⎣                  -L⋅g⋅m₃⋅cos(θ₁(t) + θ₂(t) + θ₃(t))                   ⎦

## 7.

In [21]:
G = Matrix.vstack(G1,G2,G3)
G_real = lambdify([theta_1,theta_2,theta_3], G)
G_real(np.pi/2,0,0)

array([[-6.12323399573677e-17*L*g*m_1],
       [0],
       [0],
       [1.0*L*g*m_2],
       [-6.12323399573677e-17*L*g*m_2],
       [0],
       [2.0*L*g*m_3],
       [1.0*L*g*m_3],
       [-6.12323399573677e-17*L*g*m_3]], dtype=object)

## 3 Required and optional paper review

It was an interesting reading, where the tone was a lot less formal then the previous papers we had been exposed to. I'd be curious to know if this kind of tone is something that we could do for the final paper.